# MULTIPLE POLYNOMIAL REGRESSION

<b>The data set used is the data set containing the real estate market information. It includes the number of rooms, the age of the building, the number of floors, square meters, and the price information.